In [1]:
#pip install mysql-connector-python

In [3]:
import pandas as pd
import mysql
import mysql.connector

In [5]:
dir(mysql)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'connector']

CREAR BASE DE DATOS

In [7]:
database = "yfinance_stocks"

def create_database(database, host = "localhost", user = "root", password = "Hackaboss_2024"):
    
    db = mysql.connector.connect(host     = host,
                                 user     = user,
                                 password = password)

    cursor = db.cursor()
    
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")

create_database("yfinance_stocks")

In [9]:
database = "yfinance_stocks"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "Hackaboss_2024",
                             database = database)

cursor = db.cursor()

#tabla stocks
cursor.execute('''
    CREATE TABLE stocks (
        ticker VARCHAR(10) PRIMARY KEY,
        name VARCHAR(255),
        sector VARCHAR(255),
        industry VARCHAR(255),
        market_cap BIGINT,
        full_time_employees INT,
        extraction_timestamp TIMESTAMP,
        extraction_id INT
    )
''')

#tabla historical_Prices
cursor.execute('''
    CREATE TABLE historical_prices (
        date DATE,
        ticker VARCHAR(10),
        open DECIMAL(15, 4),
        high DECIMAL(15, 4),
        low DECIMAL(15, 4),
        close DECIMAL(15, 4),
        adj_close DECIMAL(15, 4),
        volume BIGINT,
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (date, ticker),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

#tabla dividends
cursor.execute('''
    CREATE TABLE dividends (
        date DATE,
        ticker VARCHAR(10),
        dividends DECIMAL(10, 4),
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (date, ticker),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

#tabla Splits
cursor.execute('''
    CREATE TABLE splits (
        date DATE,
        ticker VARCHAR(10),
        stock_splits DECIMAL(10, 4),
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (date, ticker),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

#tabla balance_sheet
#Los Liabilities (pasivos totales) son las deudas y obligaciones combinadas que una persona o empresa tiene con terceros. 
#Todo lo que posee la empresa se clasifica como activo y todos los montos que la empresa debe por obligaciones futuras se registran como pasivos. 
#En el balance general, los activos totales menos los pasivos totales equivalen al capital.

cursor.execute('''
    CREATE TABLE balance_Sheet (
        date DATE,
        ticker VARCHAR(10),
        ordinary_shares_number BIGINT,
        net_debt BIGINT,
        total_assets BIGINT,
        current_liabilities BIGINT,
        stockholders_equity BIGINT,
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (date, ticker),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

#tabla income_statement
#El EBITDA es un indicador financiero que muestra las ganancias de una empresa, 
#antes de restar pagos y costes como impuestos, intereses, depreciaciones y amortizaciones.
#Un EPS diluido alto significa que la empresa genera muchas ganancias por acción, se utilizan 
#para comparar el EPS de una empresa con sus competidores.
#El Operating_Income es sinónimo de beneficio operativo, permite analizar 
#el rendimiento operativo de una empresa eliminando intereses e impuestos.

cursor.execute('''
    CREATE TABLE income_statement (
        date DATE,
        ticker VARCHAR(10),
        ebitda BIGINT,
        total_revenue BIGINT,
        net_income BIGINT,
        operating_income BIGINT,
        diluted_eps DECIMAL(10, 4),
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (date, ticker),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

#tabla cashflow
#Capital Expenditures (gastos de capital) y se refiere a las inversiones que realiza una empresa para 
#adquirir, mejorar o mantener activos a largo plazo como edificios, terrenos, maquinaria o equipos.
#El flujo de efectivo operativo mide el efectivo generado por las operaciones comerciales de una empresa. 
#El flujo de efectivo libre es el efectivo que una empresa genera a partir de sus operaciones comerciales 
#después de restar los gastos de capital.

cursor.execute('''
    CREATE TABLE cashflow (
        date DATE,
        ticker VARCHAR(10),
        operating_cash_flow BIGINT,
        capital_expenditure BIGINT,
        free_cash_flow BIGINT,
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (date, ticker),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

#tabla upgrades_downgrades
cursor.execute('''
    CREATE TABLE upgrades_downgrades (
        grade_date DATE,
        ticker VARCHAR(10),
        firm VARCHAR(255),
        to_grade VARCHAR(50),
        from_grade VARCHAR(50),
        action VARCHAR(50),
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (grade_date, ticker),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

#tabla holders
cursor.execute('''
    CREATE TABLE holders (
        date_reported DATE,
        holder VARCHAR(255),
        ticker VARCHAR(10),
        pct_held DECIMAL(5, 2),
        shares BIGINT,
        value BIGINT,
        extraction_timestamp TIMESTAMP,
        extraction_id INT,
        PRIMARY KEY (date_reported, holder),
        FOREIGN KEY (ticker) REFERENCES stocks(ticker)
    )
''')

db.commit()
cursor.close()
db.close()

RELLENAR BASE DE DATOS

In [17]:
database = "yfinance_stocks"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "Hackaboss_2024",
                             database = database)

cursor = db.cursor()

#función para insertar datos en la tabla stocks
def insert_stocks_data(csv_file):
    stocks_df = pd.read_csv(csv_file)

    for index, row in stocks_df.iterrows():
        cursor.execute('''
            INSERT INTO Stocks (ticker, name, sector, industry, market_cap, full_time_employees, extraction_timestamp, extraction_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            name=VALUES(name), 
            sector=VALUES(sector), 
            industry=VALUES(industry), 
            market_cap=VALUES(market_cap),
            full_time_employees=VALUES(full_time_employees), 
            extraction_timestamp=VALUES(extraction_timestamp), 
            extraction_id=VALUES(extraction_id)
        ''', #nombres del csv
            (row['ticker'], 
              row['name'], 
              row['sector'], 
              row['industry'], 
              row['marketCap'], 
              row['fullTimeEmployees'], 
              row['Extraction_Timestamp'], 
              row['Extraction_ID']))


#llamada a la función para cargar los datos del csv 
insert_stocks_data("stocks_20241021.csv")

db.commit()
cursor.close()
db.close()